# 📊 Regression from Scratch  

This notebook demonstrates how to build **linear regression models step by step, without relying on machine learning libraries**.  
The goal is to understand the mathematics and algorithms behind regression by directly implementing them in Python and NumPy.  

We will cover:  
- Ordinary Least Squares (OLS) regression  
- Coordinate descent updates  
- The role of residuals and projections  
- Extensions to regularized regression (Lasso, Ridge)  

By the end, you’ll have a clear intuition of how regression works "under the hood", beyond just calling `sklearn.LinearRegression`.  

---

👨‍💻 *Written by:* **Samuel Boluwatife Giwa**  

The solution to OLS has a closed-form expression:

$$
\hat{\beta} = (X^\top X)^{-1} X^\top y
$$


Once the coefficients are estimated, predictions are given by:

$$
\hat{y} = X \hat{\beta}
$$


## 🔹 Coefficient estimation using the Coordinate Descent Idea  

Instead of updating all coefficients at once, **we update one coordinate (one parameter) at a time**, cycling through them:  

1. **Fix all coefficients except** $ \beta_j $.  
2. **Minimize the cost function** with respect to only $ \beta_j $. 
3. **Move to the next coordinate**, and repeat.  

👉 So, instead of taking a “big step” in \( p \)-dimensional space, we take **1D steps along coordinate axes**.  


## 🔹 Coordinate Descent Update Rule  

For each parameter $ \beta_j $, keeping all others fixed:

1. **Partial residual:**

$$
r_j = y - X \beta + X_j \beta_j
$$

2. **Correlation with residual:**

$$
\tilde{\beta}_j = \frac{1}{n} X_j^\top r_j
$$

3. **Normalization factor:**

$$
a_j = \frac{1}{n} X_j^\top X_j
$$

4. **Update coefficient:**

- **OLS (no penalty):**

$$
\beta_j \leftarrow \frac{1}{a_j} \, \tilde{\beta}_j
$$

- **Lasso (with L1 penalty):**

$$
\beta_j \leftarrow \frac{1}{a_j} \, S(\tilde{\beta}_j, \lambda)
$$

where  

$$
S(z, \lambda) = \text{sign}(z) \cdot \max(|z| - \lambda, \, 0)
$$

is the **soft-thresholding operator**.


In [ ]:

# ORDINARY LEAST SQUARE

import numpy as np


# Synthetic Dataset
np.random.seed(42)

n, p = 100, 3   # 100 samples, 3 features
X = np.random.randn(n, p)   # random features
true_beta = np.array([2.0, -3.5, 1.0])  # true coefficients
y = X @ true_beta + 0.5 * np.random.randn(n)  


# Train/Test Split
indices = np.arange(n)
np.random.shuffle(indices)

train_size = int(0.7 * n)
train_idx, test_idx = indices[:train_size], indices[train_size:]

X_train, y_train = X[train_idx], y[train_idx]
X_test, y_test   = X[test_idx], y[test_idx]

n_train, _ = X_train.shape

# Training via Coordinate Descent (OLS)
beta = np.zeros(p)
iter_ = 200

for _ in range(iter_):
    for j in range(p):
        # partial residual
        r_j = y_train - X_train @ beta + X_train[:, j] * beta[j]
        
        # correlation
        beta_tilde = (1/n_train) * np.dot(X_train[:, j], r_j)
        
        a_j = (1/n_train) * np.dot(X_train[:, j], X_train[:, j])
        
    
        beta[j] = (1/a_j) * beta_tilde

# Intercept
intercept_ = y_train.mean() - np.dot(X_train.mean(axis=0), beta)

# Predictions
y_train_pred = X_train @ beta + intercept_
y_test_pred  = X_test @ beta + intercept_


# Mean Squared Error
mse_train = np.mean((y_train - y_train_pred)**2)
mse_test  = np.mean((y_test - y_test_pred)**2)

print("True coefficients:", true_beta)
print("Estimated coefficients:", beta)
print("Intercept:", intercept_)
print("Train MSE:", mse_train)
print("Test MSE:", mse_test)


True coefficients: [ 2.  -3.5  1. ]
Estimated coefficients: [ 1.89943376 -3.57613286  0.9505738 ]
Intercept: 0.062398939476852266
Train MSE: 0.192187794681832
Test MSE: 0.19902017464291


In [3]:

import numpy as np

# LASSO REGRESSION
def S(z, lam):
    return np.sign(z) * np.maximum(np.abs(z) - lam, 0.0)

np.random.seed(42)

n, p = 100, 3
X = np.random.randn(n, p)
true_beta = np.array([2.0, -3.5, 1.0])
y = X @ true_beta + 0.5 * np.random.randn(n)

indices = np.arange(n)
np.random.shuffle(indices)

train_size = int(0.7 * n)
train_idx, test_idx = indices[:train_size], indices[train_size:]

X_train, y_train = X[train_idx], y[train_idx]
X_test, y_test   = X[test_idx], y[test_idx]

n_train, p = X_train.shape
beta = np.zeros(p)
lam = 0.5
iter_ = 100

for _ in range(iter_):
    for j in range(p):
        r_j = y_train - X_train @ beta + X_train[:, j] * beta[j]
        beta_tilde = (1/n_train) * np.dot(X_train[:, j], r_j)
        a_j = (1/n_train) * np.dot(X_train[:, j], X_train[:, j])
        beta[j] = (1/a_j) * S(beta_tilde, lam)

intercept_ = y_train.mean() - np.dot(X_train.mean(axis=0), beta)

y_pred = X_test @ beta + intercept_

mse = np.mean((y_test - y_pred) ** 2)
r2 = 1 - np.sum((y_test - y_pred) ** 2) / np.sum((y_test - y_test.mean()) ** 2)

print("true coefficients:", true_beta)
print("Estimated coefficients:", beta)
print("Intercept:", intercept_)
print("Test MSE:", mse)
print("Test R2:", r2)


true coefficients: [ 2.  -3.5  1. ]
Estimated coefficients: [ 1.20034496 -3.20705442  0.55732047]
Intercept: 0.30701451085180265
Test MSE: 0.9109100636250651
Test R2: 0.9350604084826118


# Ridge Regression: Coordinate Descent Update

### 1. Objective Function
Ridge regression solves

$$
\min_{\beta}\; \frac{1}{2n}\|y - X\beta\|_2^2 + \frac{\lambda}{2}\|\beta\|_2^2,
$$




- $n$: number of samples  
- $p$: number of features  
- $X \in \mathbb{R}^{n \times p}$: design matrix  
- $y \in \mathbb{R}^{n}$: response vector  
- $\beta \in \mathbb{R}^{p}$: coefficient vector  
- $\lambda \geq 0$: regularization parameter  


---

### 2. Coordinate Descent Setup
When updating one coordinate $ \beta_j $ at a time (others fixed), define the **partial residual**

$$
r_j = y - \sum_{k\ne j} X_k \beta_k = y - X\beta + X_j\beta_j,
$$

where $ X_j $ denotes the \(j\)-th column of $ X $.  

---

### 3. Loss as a Function of $ \beta_j $
The objective restricted to $ \beta_j$ is

$$
L(\beta_j) = \frac{1}{2n}\|r_j - X_j\beta_j\|_2^2 + \frac{\lambda}{2}\beta_j^2.
$$

Expanding,

$$
L(\beta_j) = \frac{1}{2n}\!\left(r_j^T r_j - 2\beta_j X_j^T r_j + \beta_j^2 X_j^T X_j\right) + \frac{\lambda}{2}\beta_j^2.
$$

---

### 4. Derivative and Optimality Condition
Differentiate w.r.t. \(\beta_j\):

$$
\frac{\partial L}{\partial \beta_j} = -\frac{1}{n}X_j^T r_j + \frac{1}{n}\beta_j X_j^T X_j + \lambda \beta_j.
$$

Set the derivative to zero:

$$
-\frac{1}{n}X_j^T r_j + \left(\frac{1}{n}X_j^T X_j + \lambda\right)\beta_j = 0.
$$

---

### 5. Coordinate Update Rule
Solve for $ \beta_j $:

$$
\beta_j = \frac{\frac{1}{n} X_j^T r_j}{\frac{1}{n} X_j^T X_j + \lambda}.
$$

If features are standardized so that $\left(\frac{1}{n} X_j^{T} X_j = 1\right)$, this simplifies to

$$
\beta_j = \frac{\frac{1}{n} X_j^T r_j}{1 + \lambda}.
$$

---



In [5]:
import numpy as np

np.random.seed(42)

n, p = 100, 3
X = np.random.randn(n, p)
true_beta = np.array([2.0, -3.5, 1.0])
y = X @ true_beta + 0.5 * np.random.randn(n)

indices = np.arange(n)
np.random.shuffle(indices)

train_size = int(0.7 * n)
train_idx, test_idx = indices[:train_size], indices[train_size:]

X_train, y_train = X[train_idx], y[train_idx]
X_test, y_test   = X[test_idx], y[test_idx]

n_train, p = X_train.shape
beta = np.zeros(p)
lam = 0.5
iter_ = 100

for _ in range(iter_):
    for j in range(p):
        r_j = y_train - X_train @ beta + X_train[:, j] * beta[j]
        beta_tilde = (1/n_train) * np.dot(X_train[:, j], r_j)
        a_j = (1/n_train) * np.dot(X_train[:, j], X_train[:, j]) + lam
        beta[j] = beta_tilde / a_j

intercept_ = y_train.mean() - np.dot(X_train.mean(axis=0), beta)

y_train_pred = X_train @ beta + intercept_
y_test_pred = X_test @ beta + intercept_

mse_train = np.mean((y_train - y_train_pred) ** 2)
mse_test = np.mean((y_test - y_test_pred) ** 2)

r2_train = 1 - np.sum((y_train - y_train_pred) ** 2) / np.sum((y_train - y_train.mean()) ** 2)
r2_test = 1 - np.sum((y_test - y_test_pred) ** 2) / np.sum((y_test - y_test.mean()) ** 2)

print("True Beta:", true_beta)
print("Beta:", beta)
print("Intercept:", intercept_)
print("Train MSE:", mse_train, " Test MSE:", mse_test)
print("Train R^2:", r2_train, " Test R^2:", r2_test)


True Beta: [ 2.  -3.5  1. ]
Beta: [ 1.19661734 -2.47528447  0.72070361]
Intercept: 0.4120446108206335
Train MSE: 1.8920924188714143  Test MSE: 1.5336362760345037
Train R^2: 0.8928163011957736  Test R^2: 0.8906657009523147
